# Comparing datasets

Datasets:
- iDesignRES / OpenEntrance - aggregated (denoted by o_\<something\>)
- pyPSA-Eur - non-aggregated. (denoted by p_\<something\>)

Goals:
- Check the difference of the energy mixes throughout the whole dataset.

Spain:
- What is the energy mix of Spain
- What power generating capacity does Spain have?
- What loads does Spain have?


Questions:
What is the difference between primary and secondary energy?

In [2]:
import pandas as pd

First, we want to **plot the energy mix in OpenEntrance:**

In [7]:
IAMC_annual_data = pd.read_csv('../networks/OpenEntrance/IAMC_annual_data.csv')

# Select just Region that contains Spain
IAMC_annual_data_ES = IAMC_annual_data[IAMC_annual_data['Region'].str.contains('Spain', case=False, na=False)]
IAMC_annual_data_ES

# 

,Model,Scenario,Region,Variable,Unit,2030
111,GENeSYS-MOD 3.1,Techno-Friendly 2.0,France>Spain,Network|Electricity|Maximum Flow,MW,5.000000e+03
217,GENeSYS-MOD 3.1,Techno-Friendly 2.0,Portugal>Spain,Network|Electricity|Maximum Flow,MW,4.500000e+03
227,GENeSYS-MOD 3.1,Techno-Friendly 2.0,Spain,Capacity|Electricity|Biomass|w/o CCS,MW,1.747281e+03
228,GENeSYS-MOD 3.1,Techno-Friendly 2.0,Spain,Capacity|Electricity|Coal|Hard coal|w/o CCS,MW,3.946900e+03
229,GENeSYS-MOD 3.1,Techno-Friendly 2.0,Spain,Capacity|Electricity|Coal|Lignite|w/o CCS,MW,3.600000e+02
...,...,...,...,...,...,...
296,GENeSYS-MOD 3.1,Techno-Friendly 2.0,Spain,Variable Cost|Electricity|Wind|Onshore,EUR/MWh,3.600000e-02
297,GENeSYS-MOD 3.1,Techno-Friendly 2.0,Spain>France,Network|Electricity|Maximum Flow,MW,5.000000e+03
298,GENeSYS-MOD 3.1,Techno-Friendly 2.0,Spain>Portugal,Network|Electricity|Maximum Flow,MW,4.100000e+03
322,plan4eu-V1.1,ENTSOETransparency,Spain,Maximum Storage|Electricity|Hydro|Reservoir,MWh,1.640904e+07
